## Redes Neuronales
#### Dataset "Diabetes"

Hacemos las importaciones de librerias para la red neuronal

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

En esta sección podremos definir los hiper parametros con los que va a trabajar la red neuronal

In [28]:
# Hiperparámetros
hidden_layer_sizes = (100, 100, 100)
batch_size = 64
learning_rate_init = 0.001
max_iter = 10
random_state = 42

Posteriormente cargamos el CSV de Diabetes y definimos la cantidad de datos en el train y en el test

In [29]:
# Cargar los datos
df = pd.read_csv('Diabetes.csv')

# Dividir los datos en características (X) y etiquetas (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Submuestreo: Definirles la media a los datos con Outcome igual a 0
X_train_0 = X[y == 0].sample(n=sum(y == 1), random_state=random_state)
X_train_1 = X[y == 1]
y_train_0 = pd.Series([0] * len(X_train_0), name='Outcome')
y_train_1 = pd.Series([1] * len(X_train_1), name='Outcome')

# Unir los datos de entrenamiento balanceados
X_train_balanced = pd.concat([X_train_0, X_train_1])
y_train_balanced = pd.concat([y_train_0, y_train_1])

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_train_balanced, y_train_balanced, test_size=0.2, random_state=random_state)

# Normalizar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Finalmente definimos el modelo de aprendizaje y realizamos la predicción en el conjunto de datos de test para verificar que tan predecible fue el modelo entrenado

In [30]:
# Entrenar el modelo MLPClassifier
model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,
                      batch_size=batch_size,
                      learning_rate_init=learning_rate_init,
                      max_iter=max_iter,
                      random_state=random_state)
model.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred = model.predict(X_test_scaled)

c:\Users\rolbi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


Por último, obtenemos las diferentes metricas y ver que valores nos brinda el modelo que vamos a analizarlas más a detalle

In [31]:
# Calcular métricas de evaluación
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

# Imprimir métricas de evaluación
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.7962962962962963
Precision: 0.7547169811320755
Recall: 0.8163265306122449
